In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/DijiData

/content/drive/MyDrive/DijiData


Reading Datas

In [3]:
import numpy as np
import pandas as pd

In [5]:
student_question_df = pd.read_excel("FourSchoolData.xlsx", sheet_name = "StudentQuestionAnswer")
student_question_df.head()

,StudentId,QuestionId,StudentAnswer,Date
0,65434,4861303,C,2021-10-01 11:42:17
1,65434,4861329,B,2021-10-01 11:42:17
2,65434,3318245,C,2021-10-01 11:42:17
3,65434,3318244,A,2021-10-01 11:42:17
4,65434,3318254,D,2021-10-01 11:42:17


This frame gives the data for each student answers for different questions in given time. 

In [6]:
exam_taken_df = pd.read_excel("FourSchoolData.xlsx", sheet_name = "ExamTakes")
exam_taken_df.head()

,DateTaken,ExamId,StudentId
0,2021-10-01 11:42:17,134452,65434
1,2021-10-01 11:43:07,134453,65434
2,2021-10-01 11:40:52,134461,65434
3,2021-09-21 17:46:24,134896,130678
4,2021-09-21 18:15:40,134914,130678


Each student when they are take indicated exam

In [7]:
exam_questions_df = pd.read_excel("FourSchoolData.xlsx", sheet_name = "ExamQuestions")
exam_questions_df.head()

,ExamId,QuestionId,ObjectiveId,CorrectAnswer,DifficultyLevel,ExamQuestionOrder
0,128205,3578703,3044,A,3,1
1,128205,3515998,3042,D,3,2
2,128205,3382433,3043,D,3,3
3,128205,1185417,3041,B,3,4
4,128205,1369229,3044,C,3,5


The information related with exam such as which objectives exam questionId correct answer. 

In [8]:
exam_workplan_df = pd.read_excel("FourSchoolData.xlsx", sheet_name = "Exam-WorkPlan")
exam_workplan_df.head()

,ExamId,PlanId
0,128205,4351
1,128206,4351
2,128207,4351
3,128208,4352
4,128209,4352


In [9]:
exam_characteristics_df = pd.read_excel("FourSchoolData.xlsx", sheet_name = "ExamCharacteristics")
exam_characteristics_df.head()

,ExamId,ExamName,StudentId,DERS,SORUSAYISI,CEVAP
0,128205,M.8.1.2. Üslü İfadeler Testleri 1. Test,133580,Matematik 8,10,ADDBCCBBCB
1,128206,M.8.1.2. Üslü İfadeler Testleri 2. Test,133580,Matematik 8,10,CCCDDCABBD
2,128207,M.8.1.2. Üslü İfadeler Testleri 3. Test,133580,Matematik 8,10,BABCADBCCC
3,128208,M.8.1.3. Kareköklü İfadeler Testleri 1. Test,133580,Matematik 8,9,BDBDAABBB
4,128209,M.8.1.3. Kareköklü İfadeler Testleri 2. Test,133580,Matematik 8,10,CCBCADCBDD


In [11]:
student_workplan_df = pd.read_excel("FourSchoolData.xlsx", sheet_name = "Student-WorkPlan")
student_workplan_df.head()

,PlanId,StudentId
0,4351,133580
1,4352,133580
2,4353,133580
3,4354,133580
4,4355,133580


In [12]:
student_class_df = pd.read_excel("FourSchoolData.xlsx", sheet_name = "Students-Class")
student_class_df.head()

,ClassId,StudentId,SchoolId,GradeLevel,PercentileGoal
0,3276,56108,431,12,NaN
1,3276,56111,431,12,NaN
2,2537,59218,348,8,NaN
3,2537,59241,348,8,NaN
4,3496,59430,352,12,NaN


In [33]:
schools_df = pd.read_excel("FourSchoolData.xlsx", sheet_name = "Schools")
schools_df

,SchoolId,Type,Name
0,413,148,UZEMGO GÜRSEL ONLİNE UZAKTAN EĞİTİM
1,2250,213,Fatsa Çocuk Kulubü
2,2268,209,DÖRT RENK OKULLARI
3,2722,209,AÇI ÖZEL ÖĞRETİM KURSU
4,3815,209,BAŞAKŞEHİR ÖZEL FEN BİLİMLERİ


In the student_class_df, there are some students from different schools. We need to determine how many students are there from different schools and we treat them accordingly. 

In [34]:
school_count_frame = pd.DataFrame(student_class_df.sort_values(by = "SchoolId").groupby("SchoolId")["StudentId"].count())
school_count_frame.rename({"StudentId": "Count"}, axis = "columns", inplace = True)
school_count_frame

,Count
SchoolId,
348,12
352,7
413,1222
431,2
522,1
562,1
2250,1
2268,1152
2722,123


In [35]:
school_count_frame = school_count_frame.merge(schools_df, how = "outer", left_index = True, right_on = "SchoolId")
school_count_frame.set_index("SchoolId", inplace = True)
school_count_frame.drop(columns = "Type", axis = 1, inplace = True)
school_count_frame

,Count,Name
SchoolId,,
348,12,NaN
352,7,NaN
413,1222,UZEMGO GÜRSEL ONLİNE UZAKTAN EĞİTİM
431,2,NaN
522,1,NaN
562,1,NaN
2250,1,Fatsa Çocuk Kulubü
2268,1152,DÖRT RENK OKULLARI
2722,123,AÇI ÖZEL ÖĞRETİM KURSU


In [53]:
unknown_school_ids = (school_count_frame[school_count_frame["Name"].isna()]).index
unknown_school_ids

Int64Index([348, 352, 431, 522, 562], dtype='int64', name='SchoolId')

Found unwanted schoolIds now we need to filter student_class_df frame. Also we may want to exclude the data of Fatsa Çocuk Kulubü because only 1 person exist in the data. 

In [58]:
filter_for_unwanted_sch = ~student_class_df.SchoolId.isin(unknown_school_ids)
student_class_df = student_class_df[filter_for_unwanted_sch]
student_class_df

,ClassId,StudentId,SchoolId,GradeLevel,PercentileGoal
15,3621,65434,413,7,1.0
16,11106,65434,413,8,1.0
17,11107,65434,413,7,1.0
26,3136,83763,413,8,NaN
27,3136,83939,413,8,NaN
...,...,...,...,...,...
2923,27364,329189,413,5,NaN
2924,3137,329190,413,9,NaN
2925,27560,329191,413,8,NaN
2926,3553,329209,413,10,NaN
